In [11]:
import json
from pprint import pprint

### Read the year wise paper json files

In [6]:
def read_papers_json(year):
    paper_data = []
    
    file_path = "./science-parse/output/{}/{}_ICLR".format(year, year)
    with open(file_path, "r") as f:
        for line in f:
            paper_data.append(json.loads(line))
            
    return paper_data

In [7]:
paper_data = read_papers_json(2017)

In [12]:
pprint(paper_data[-1])

{'metadata': {'abstractText': 'Recent work has begun exploring neural acoustic '
                              'word embeddings—fixeddimensional vector '
                              'representations of arbitrary-length speech '
                              'segments corresponding to words. Such '
                              'embeddings are applicable to speech retrieval '
                              'and recognition tasks, where reasoning about '
                              'whole words may make it possible to avoid '
                              'ambiguous sub-word representations. The main '
                              'idea is to map acoustic sequences to '
                              'fixed-dimensional vectors such that examples of '
                              'the same word are mapped to similar vectors, '
                              'while different-word examples are mapped to '
                              'very different vectors. In this work we take a '
   

In [16]:
print(paper_data[0].keys())
print(paper_data[0]['metadata'].keys())

dict_keys(['metadata', 'name'])
dict_keys(['creator', 'sections', 'authors', 'referenceMentions', 'title', 'source', 'abstractText', 'emails', 'references', 'year'])


### Construct features

In [ ]:
def construct_features()